In [4]:
#!/usr/bin/env sage -python

import time
import random
import hashlib
import sys
from sage.all import *

class RSA:
    def get_bits(self, a):
        n=1
        while a!=1:
            n+=1
            a//=2
        return n
    BlockLens=0 # Размер блока, который шифруется
    '''some public class members'''
    e = 0
    N = 0
    '''some private class members'''
    __d = 0
    __p = 0
    __q = 0
    __fN = 0
    __min_bits_length = 129
    
    def __my_init(self):
        start = time.time()
        self.e = 0x111113
        #self.e = 0x117
        self.__p, self.__q = self.__gen_next_primes()
        self.N = self.__p * self.__q
        self.__fN = self.__gen_phi()
        self.__d = self.__gen_private_exponent()
        end = time.time()
        self.print_in_files() # Сохраняем ключи в файлах
        self.BlockLens=(self.get_bits(self.N)-1) // 8 # Размер блока для шифрования
        print("BlockLens=", self.BlockLens)
        self.print_debug()
        print(end - start)
    def __init__(self, param=0):
        if param==0: # Режим создания всех ключей
           self.__my_init() 
        if param==1: # Режим шифрования или проверки электронной подписи
           self.read_public_key() # Прочитать из файла открытый ключ
           self.BlockLens=(self.get_bits(self.N)-1) // 8
           self.print_debug()
        if param==2: # Режим расшифрования
           self.read_private_key() # Прочитать из файла закрытый ключ
           self.BlockLens=(self.get_bits(self.N)-1) // 8        
        if param==3: # Режим электронной подписи
           self.read_private_key() # Прочитать из файл закрытый ключ
           self.BlockLens=(self.get_bits(self.N)-1) // 8        
         
    def __gen_next_primes(self):
        return random_prime(2^self.__min_bits_length - 1, False, 2^(self.__min_bits_length - 1)), random_prime(2^self.__min_bits_length - 1, False, 2^(self.__min_bits_length - 1))
    
    def __gen_phi(self):
        return (self.__p - 1) * (self.__q - 1)
        #return euler_phi(self.N)
    
    def __gen_private_exponent(self):
        return power_mod(self.e, -1, self.__fN)
    
    def RSA_encrypt(self, plain):
        return power_mod(plain, self.e, self.N) #plain has to be an integer
    
    def RSA_decrypt(self, cipher):
        return power_mod(cipher, self.__d, self.N)
    
    def RSA_sign(self, message):
        return power_mod(message, self.__d, self.N) #message is also integer
    
    def print_debug(self):
        print("Debug info:\n p = {0},\n q = {1},\n N = {2},\n d = {3},\n e= {4},\n F(N) = {5}".format(hex(self.__p), hex(self.__q), hex(self.N), hex(self.__d),hex(self.e), hex(self.__fN)))
    def print_in_files(self): # Сохранение ключей в файлах
        myF1=open("public_key.rsa", "w")
        myF1.write(self.e.str(10)+"\n")
        myF1.write(self.N.str(10))
        myF1.close()
        myF1=open("private_key.rsa", "w")
        myF1.write(self.__d.str(10)+"\n")
        myF1.write(self.N.str(10))
        myF1.close()
    def read_private_key(self): # Чтение секретного ключа
        myF1=open("private_key.rsa", "r")
        i=0
        for str in myF1:
            if (i==0):
                s=str[0:-1]
                self.__d=Integer(s)
            else:
                self.N=Integer(str)
            i+=1
        myF1.close()
    def read_public_key(self): # Чтение закрытого ключа
        myF1=open("public_key.rsa", "r")
        i=0
        for str in myF1:
            if (i==0):
                s=str[0:-1]
                self.e=Integer(s)
            else:
                self.N=Integer(str)
            i+=1
        myF1.close()
class MYRSA_sign: # Класс для электронной подписью
# Перевод последовательности байт в целое число
	def bytes_to_int(self, bytes):
		rez=0
		for b in bytes:
			rez=rez*256+int(b)
		return rez

	def __init__(self, param=0):
		self.rsa=RSA(param) # Объект rsa

	def hesh(self, file): # Получение хеша файла
		SIZE = 65536 #  Максимальный размер блока файла
		hasher = hashlib.new('sha256')
		with open(file, 'rb') as f:
			buffer = f.read(SIZE)
			while len(buffer) > 0:
				hasher.update(buffer)
				buffer = f.read(SIZE)
		return hasher.digest() # Хеш в виде последовательности байтов
	def file_sign(self, file1, file_sign): # Для подписывания файла
		h=self.hesh(file1)
		print("Хеш: ", h)
		h2=self.rsa.RSA_sign(self.bytes_to_int(h)) # Преобразуем последовательность байтов в целое в шифруем закрытым ключом
		print("Шифр Хеша: ", h2)
		myF1=open(file_sign, "wb")
		bytes=int(h2).to_bytes(self.rsa.BlockLens+1, byteorder='big') # Целое число в массив байт размерности rsa.BlockLens+1
		print("Массив шифра Хеша: ", bytes)
		myF1.write(bytes); # Подпись сохраняем в файле
		myF1.close()
	def proverka_sign(self, file1, file_sign): # Проверка подписи
		h=self.hesh(file1) # Получаем хеш файла
		myF1=open(file_sign, "rb") # Файл с подписью
		bytes=myF1.read(self.rsa.BlockLens+1)
		h2=self.rsa.RSA_encrypt(self.bytes_to_int(bytes)) # Последовательность байт в целое и расшифровываем
		bytes=int(h2).to_bytes(self.rsa.BlockLens, byteorder='big') # Целое число в массив байт размерности rsa.BlockLens+1
		myF1.close()
		print(bytes)
		print(h)
		if h==bytes:
			return True
		else:
			return False
		
		

		
mySign1 = MYRSA_sign() # Режим создания всех ключей и сохранения в файле
# Получение подписи файла
mySign1.file_sign("C:/Users/Директор/Desktop/STUD/ТиМП/Лабораторная работа 4.docx", "C:/Users/Директор/Desktop/STUD/ТиМП//sign.sig");
# Проверка подписи файла
mySign2 = MYRSA_sign(1)
if mySign2.proverka_sign("C:/Users/Директор/Desktop/STUD/ТиМП/Лабораторная работа 4.docx", "C:/Users/Директор/Desktop/STUD/ТиМП/sign.sig")==True:
	print("Подпись подлинна")
else:
	print("Подпись не подлинна")


ModuleNotFoundError: No module named 'sage'